# Snorkel Workshop: Extracting Spouse Relations from the News

## Part 4: Hyperparameter Tuning via Grid Search


In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import numpy as np

# Connect to the database backend and initalize a Snorkel session
from lib.init import *

We repeat our definition of the `Spouse` `Candidate` subclass, and load the test set:

In [2]:
Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

# 1 Training a `SparseLogReg` Discriminative Model
We use the training marginals to train a discriminative model that classifies each `Candidate` as a true or false mention. We'll use a random hyperparameter search, evaluated on the development set labels, to find the best hyperparameters for our model. To run a hyperparameter search, we need labels for a development set. If they aren't already available, we can manually create labels using the Viewer.

## Feature Extraction
Instead of using a deep learning approach to start, let's look at a standard sparse logistic regression model. First, we need to extract out features. This can take a while, but we only have to do it once!

In [3]:
from snorkel.annotations import FeatureAnnotator
featurizer = FeatureAnnotator()

In [4]:
F_train = featurizer.load_matrix(session, split=0)
F_dev = featurizer.load_matrix(session, split=1)
F_test = featurizer.load_matrix(session, split=2)

# #if F_train.size == 0:    
# %time F_train = featurizer.apply(split=0, parallelism=4)
# #if F_dev.size == 0:     
# %time F_dev  = featurizer.apply_existing(split=1, parallelism=4)
# #if F_test.size == 0:
# %time F_test = featurizer.apply_existing(split=2, parallelism=4)

print F_train.shape
print F_dev.shape
print F_test.shape

(22254, 547350)
(2811, 547350)
(2701, 547350)




First, reload the training marginals:

In [5]:
from snorkel.annotations import load_marginals
train_marginals = load_marginals(session, split=0)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(train_marginals, bins=20)
plt.show()

In [6]:
from snorkel.learning import SparseLogisticRegression
disc_model = SparseLogisticRegression()

The following code performs model selection by tuning our learning algorithm's hyperparamters.

In [7]:
from snorkel.learning.utils import MentionScorer
from snorkel.learning import RandomSearch, ListParameter, RangeParameter

# Searching over learning rate
rate_param = RangeParameter('lr', 1e-6, 1e-2, step=1, log_base=10)
l1_param  = RangeParameter('l1_penalty', 1e-6, 1e-2, step=1, log_base=10)
l2_param  = RangeParameter('l2_penalty', 1e-6, 1e-2, step=1, log_base=10)

searcher = RandomSearch(session, disc_model, 
                        F_train, train_marginals, 
                        [rate_param, l1_param, l2_param], 
                        n=4)

Initialized RandomSearch search of size 4. Search space size = 125.


In [8]:
from snorkel.annotations import load_gold_labels

L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_dev.shape

(2811, 1)

In [9]:
np.random.seed(1701)
searcher.fit(F_dev, L_gold_dev, n_epochs=400, rebalance=0.5, print_freq=25)

[1] Testing lr = 1.00e-02, l1_penalty = 1.00e-02, l2_penalty = 1.00e-04
[SparseLogisticRegression] Training model
[SparseLogisticRegression] n_train=5252  #epochs=400  batch size=256
[SparseLogisticRegression] Epoch 0 (0.51s)	Average loss=438.817993
[SparseLogisticRegression] Epoch 25 (13.13s)	Average loss=442.243896
[SparseLogisticRegression] Epoch 50 (26.21s)	Average loss=442.291931
[SparseLogisticRegression] Epoch 75 (38.79s)	Average loss=442.853973
[SparseLogisticRegression] Epoch 100 (51.16s)	Average loss=443.277283
[SparseLogisticRegression] Epoch 125 (63.12s)	Average loss=443.277100
[SparseLogisticRegression] Epoch 150 (75.71s)	Average loss=443.391602
[SparseLogisticRegression] Epoch 175 (88.29s)	Average loss=443.762268
[SparseLogisticRegression] Epoch 200 (101.26s)	Average loss=443.870178
[SparseLogisticRegression] Epoch 225 (114.11s)	Average loss=444.545288
[SparseLogisticRegression] Epoch 250 (127.38s)	Average loss=444.714417
[SparseLogisticRegression] Epoch 275 (140.65s)	Ave

,lr,l1_penalty,l2_penalty,Prec.,Rec.,F1
0,0.0100,0.010000,0.000100,0.536082,0.273684,0.362369
2,0.0010,0.000001,0.000100,0.256198,0.326316,0.287037
1,0.0001,0.000001,0.000001,0.233083,0.326316,0.271930
3,0.0010,0.010000,0.001000,0.247664,0.278947,0.262376


## Examining Features
Extracting features allows us to inspect and interperet our learned weights 

In [16]:
w, _ = disc_model.get_weights()
largest_idxs = list(reversed(np.argsort(np.abs(w))[-50:]))

pos_ftrs = [(F_train.get_key(session, i).name, w[i]) for i in largest_idxs if w[i] > 0.0]
neg_ftrs = [(F_train.get_key(session, i).name, w[i]) for i in largest_idxs if w[i] <= 0.0]

pos_ftrs = sorted(pos_ftrs, key=lambda x:x[-1], reverse=1)
neg_ftrs = sorted(neg_ftrs, key=lambda x:x[-1], reverse=1)

for ftr in pos_ftrs:
    print('Feature: {0: <90}Weight: {1:.6f}'.format(*ftr))
print "-" * 90
for ftr in neg_ftrs:
    print('Feature: {0: <90}Weight: {1:.6f}'.format(*ftr))

Feature: TDL_INV_LEMMA:RIGHT-OF-MENTION['s actress]                                                Weight: 0.728911
Feature: TDL_INV_DEP_LABEL|LEMMA:BETWEEN-MENTION-and-MENTION[ROOT|write prep|on]                   Weight: 0.708714
Feature: TDL_INV_LEMMA:BETWEEN-MENTION-and-MENTION[bruk]                                           Weight: 0.659242
Feature: TDL_INV_LEMMA:SEQ-BETWEEN[actress daughter]                                               Weight: 0.615915
Feature: TDL_LEMMA:PARENTS-OF-BETWEEN-MENTION-and-MENTION[that lynxx leader]                       Weight: 0.573827
Feature: TDL_INV_LEMMA:RIGHT-OF-MENTION[von before]                                                Weight: 0.567666
Feature: TDL_INV_LEMMA:RIGHT-OF-MENTION[f.]                                                        Weight: 0.539246
Feature: TDL_DEP_LABEL|LEMMA:BETWEEN-MENTION-and-MENTION[ROOT|disagree nsubj|parent prep|with]     Weight: 0.514130
Feature: TDL_INV_LEMMA:SEQ-BETWEEN[wife]                                

## Evaluate on Test Data

In [ ]:
from snorkel.annotations import load_gold_labels
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)

In [ ]:
_, _, _, _ = disc_model.score(session, F_test, L_gold_test)